In [10]:
import tkinter as tk
from tkinter import simpledialog, scrolledtext

# Updated menu prices from USD to INR (assuming 1 USD = 75 INR)
menu = {
    "Appetizers": {"Spring Rolls": 150.00, "Garlic Bread": 100.00, "Salad": 100.00},
    "Main Course": {"Pasta": 250.00, "Burger": 200.00, "Pizza": 300.00},
    "Desserts": {"Ice Cream": 120.00, "Cake": 110.00, "Fruit Salad": 150.00},
    "Beverages": {"Coffee": 40.00, "Tea": 30.00, "Soda": 50.00}
}

order = []

def update_chat(message):
    """Update the chat box with a new message."""
    chat_box.config(state=tk.NORMAL)
    chat_box.insert(tk.END, f"{message}\n\n")
    chat_box.config(state=tk.DISABLED)
    chat_box.see(tk.END)

def clear_chat():
    """Clear the chat box."""
    chat_box.config(state=tk.NORMAL)
    chat_box.delete(1.0, tk.END)
    chat_box.config(state=tk.DISABLED)

def show_main_menu():
    """Display the main menu options."""
    clear_buttons()
    update_chat("Choose an option:")
    create_button("View Menu", view_menu, bg="light blue", fg="black")
    create_button("Order Food", select_category, bg="light green", fg="black")
    create_button("View Bill", calculate_bill, bg="light yellow", fg="black")
    create_button("Edit Order", edit_order, bg="orange", fg="black")
    create_button("Place Order", place_order, bg="light coral", fg="black")

buttons = []

def create_button(text, command, bg="white", fg="black"):
    """Create a button with specific text, command, background, and foreground color."""
    button = tk.Button(root, text=text, command=command, width=30, bg=bg, fg=fg)
    button.pack(pady=5)
    buttons.append(button)

def clear_buttons():
    """Clear all buttons from the window."""
    for button in buttons:
        button.destroy()
    buttons.clear()

def view_menu():
    """Display the menu for viewing purposes only."""
    clear_buttons()
    update_chat("Menu:")
    for category, items in menu.items():
        update_chat(f"{category}:")
        for item, price in items.items():
            update_chat(f"{item} - ₹{price:.2f}")
    show_main_menu()  # After viewing the menu, go back to the main menu

def select_category():
    """Allow user to select a category for ordering."""
    clear_buttons()  # Ensure old buttons are removed
    update_chat("Select a category:")
    category_colors = {"Appetizers": "#98FB98", "Main Course": "#32CD32", "Desserts": "#90EE90", "Beverages": "#66CDAA"}  # Different shades of green
    for category in menu.keys():
        create_button(category, lambda c=category: show_order_menu(c), bg=category_colors.get(category, "light green"), fg="black")
    
    # Add "Back to Main Menu" button to allow returning to the main menu
    create_button("Back to Main Menu", show_main_menu, bg="light blue", fg="black")

def show_order_menu(category):
    """Display the order menu for a specific category."""
    clear_buttons()
    update_chat(f"Order Menu for {category}:")
    items = menu.get(category, {})
    for item, price in items.items():
        create_button(f"Order {item} - ₹{price:.2f}", lambda i=item, p=price, c=category: add_to_order(i, p, c), bg="light pink", fg="black")
    
    create_button("Back to Select Category", select_category, bg="light green", fg="black")
    create_button("Back to Main Menu", show_main_menu, bg="light blue", fg="black")

def add_to_order(item, price, category):
    """Add item to the order and confirm addition."""
    quantity = simpledialog.askinteger("Quantity", f"How many {item} would you like to order?", minvalue=1)
    if quantity:
        for i, (existing_item, existing_price, existing_quantity) in enumerate(order):
            if existing_item == item:
                order[i] = (existing_item, existing_price, existing_quantity + quantity)
                update_chat(f"Updated {quantity}x {item} in {category}.")
                break
        else:
            order.append((item, price, quantity))
            update_chat(f"Added {quantity}x {item} in {category}.")
        
        show_order_menu(category)

def calculate_bill():
    """Calculate and display the bill."""
    if not order:
        update_chat("Your order is currently empty.")
        return
    
    total = 0
    order_details = []

    for item, price, quantity in order:
        item_total = price * quantity
        total += item_total
        order_details.append(f"{quantity}x {item} - ₹{price:.2f} each: ₹{item_total:.2f}")  # Updated currency symbol to rupee (₹)
    
    bill_summary = "Your order:\n" + "\n".join(order_details) + f"\n\nTotal: ₹{total:.2f}"
    update_chat(bill_summary)

def place_order():
    """Place the order and collect customer details."""
    if not order:
        update_chat("Your order is currently empty.")
        return
    
    # Collect customer details
    name = simpledialog.askstring("Customer Details", "Enter your name:")
    phone_number = simpledialog.askstring("Customer Details", "Enter your phone number:")
    
    if name and phone_number:
        calculate_bill()
        update_chat(f"Thank you, {name}! Your order is placed. We will call you at {phone_number} to Reach your Home.")
        order.clear()
        root.after(100000, start_chat)
    else:
        update_chat("Order placement canceled. Please provide valid details.")
        show_main_menu()

def edit_order():
    """Allow the user to edit their order, excluding confirmed items."""
    clear_buttons()
    
    if not order:
        update_chat("You have no items to edit.")
        show_main_menu()
        return

    update_chat("You can edit the following items in your order:")
    for index, (item, price, quantity) in enumerate(order):
        create_button(f"Remove item(s) from {quantity}x {item} - ₹{price:.2f} each", lambda i=index: remove_item(i), bg="orange", fg="black")
    
    create_button("Back to Main Menu", show_main_menu, bg="light blue", fg="black")

def remove_item(index):
    """Remove an item or quantity of an item from the order."""
    item, price, quantity = order[index]
    remove_quantity = simpledialog.askinteger("Remove Quantity", f"How many {item} would you like to remove?", minvalue=1, maxvalue=quantity)
    
    if remove_quantity:
        if remove_quantity < quantity:
            order[index] = (item, price, quantity - remove_quantity)
            update_chat(f"Removed {remove_quantity}x {item} from your order. {quantity - remove_quantity}x remaining.")
        else:
            order.pop(index)
            update_chat(f"Removed all {item} from your order.")
        
        edit_order()

def start_chat():
    """Start the chat by welcoming the user and displaying the main menu."""
    clear_chat()
    update_chat("Welcome to our restaurant! How can we assist you today?")
    show_main_menu()

root = tk.Tk()
root.title("Restaurant Chatbot")
root.geometry("600x800")

chat_box = scrolledtext.ScrolledText(root, wrap=tk.WORD, state=tk.DISABLED)
chat_box.pack(pady=10, padx=10, fill=tk.BOTH, expand=True)

start_chat()

root.mainloop()